In [ ]:
import pandas as pd
from rtlsdr import RtlSdr
import numpy as np

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=False)
import plotly.express as px


In [ ]:
def get_signal(center_freq=102.7e6, sample_rate=1e6):
    try:
        # close it if one was left open somewhere in the notebook
        sdr.close()
    except:
        pass
    
    # Example code from rtlsdr
    sdr = RtlSdr()
    # sdr.sample_rate = 3.2e6 # Hz
    sdr.sample_rate = sample_rate # Hz
    sdr.center_freq = center_freq   # Hz, ~100MHz
    sdr.freq_correction = 60  # PPM
    print(sdr.valid_gains_db)
    
    # sdr.gain = 49.6
    sdr.gain = "auto"
    print(sdr.gain)
    
    x = sdr.read_samples(4096)
    print(len(x))
    #help prevent the RTL-SDR from going into a glitched out state where it needs to be unplugged/replugged
    sdr.close()
    return x

s = get_signal()

In [ ]:
df = pd.DataFrame(s, columns=["signal"])

# Split the signal's complex numbers into real and imaginary components
df["signal_real"] = df["signal"].apply(lambda row: row.real)
df["signal_imag"] = df["signal"].apply(lambda row: row.imag)

print(df.shape)
df.head()

### The first 2048 samples are 'transients' and can be ignored
https://pysdr.org/content/rtlsdr.html

In [ ]:
df_plot = df.iloc[:,1:]
fig = px.scatter(df_plot, y="signal_imag", color="signal_real")
fig.show()

In [ ]:
df_plot = df.iloc[2048:,1:]
fig = px.scatter(df_plot, y="signal_imag", color="signal_real")
fig.show()

# FFT
https://pysdr.org/content/frequency_domain.html#fft-in-python

### Simple intro

In [ ]:
# time
t = np.arange(1000)
# signal
s = np.cos(t/10)
px.line(s)

In [ ]:
# signal in frequency domain
S = np.fft.fft(s)

In [ ]:
# Interpret the complex numbers as magnitude and phase
S_mag = np.abs(S)
S_phase = np.angle(S)

In [ ]:
px.line(S_mag)

### 'shift' the fft signal, because the it is formatted in a conveninent way in the numpy output
The default output is [0, -ve freqs, +ve freqs], which is dumb, we want it zero centered

In [ ]:
S = np.fft.fftshift(S)
# Interpret the complex numbers as magnitude and phase
S_mag = np.abs(S)
S_phase = np.angle(S)

In [ ]:
# So we see, apparently there are two signals... ?
px.line(S_mag)

### Using the signal

In [ ]:
center_freq=102.7e6
sample_rate=1e6

def plot_frequency_space(center_freq=102.7e6, sample_rate=1e6):
    # The output x axis ranges from -(x/2) to (x/2), where x is the sample rate, and 0 is centered on the target frequency
    # In this case, it is 102.2 MHz +/- ~2 MHz
    s = get_signal(center_freq=center_freq, sample_rate=sample_rate)
    
    # fft the input, and center it on 0
    s_fft = np.fft.fftshift(np.fft.fft(s[:2048]))
    
    dfft = pd.DataFrame(s_fft, columns=["signal_fft"])
    
    # Split the signal's complex numbers into real and imaginary components
    dfft["signal_real_fft"] = dfft["signal_fft"].apply(lambda row: row.real)
    dfft["signal_imag_fft"] = dfft["signal_fft"].apply(lambda row: row.imag)
    
    # Calculate magnitude and angle of the complex signal
    dfft["signal_mag_fft"] = dfft["signal_fft"].apply(np.abs)
    dfft["signal_angle_fft"] = dfft["signal_fft"].apply(np.angle)
    
    # Estimate frequencies 
    dfft["frequency"] = np.linspace(center_freq-(sample_rate/2), center_freq+(sample_rate/2), dfft.shape[0])
    
    fig = px.scatter(dfft, x="frequency", y="signal_mag_fft", color="signal_angle_fft")
    fig.show()

In [ ]:
plot_frequency_space(center_freq=99.7e6)

In [ ]:
dfft

In [ ]:
fig.update_xaxes(range=[0,1])

In [ ]:
# # ...
# sdr.sample_rate = 2.4e5 # Hz
# # ...

# fft_size = 512
# num_rows = 500
# x = sdr.read_samples(2048) # get rid of initial empty samples
# x = sdr.read_samples(fft_size*num_rows) # get all the samples we need for the spectrogram
# spectrogram = np.zeros((num_rows, fft_size))
# for i in range(num_rows):
#     spectrogram[i,:] = 10*np.log10(np.abs(np.fft.fftshift(np.fft.fft(x[i*fft_size:(i+1)*fft_size])))**2)
# extent = [(sdr.center_freq + sdr.sample_rate/-2)/1e6,
#             (sdr.center_freq + sdr.sample_rate/2)/1e6,
#             len(x)/sdr.sample_rate, 0]
# plt.imshow(spectrogram, aspect='auto', extent=extent)
# plt.xlabel("Frequency [MHz]")
# plt.ylabel("Time [s]")
# plt.show()